In [ ]:
import matplotlib.pyplot as plt

def plot_over_epochs(y_values: list, title: str, y_label: str, legend: str, y_values_prime: list = None, legend_prime: str = None):
    plt.figure(figsize=(4, 4))
    plt.plot(list(range(len(y_values))), y_values, label=legend)
    plt.title(title)
    plt.xlabel('epochs')
    plt.ylabel(y_label)
    plt.legend()
    plt.show()

In [ ]:
from neural_network import NeuralNetwork
from math_functions.loss import LossFunction
from math_functions.activation import ActivationFunction
from network_utility import Metrics
from dataset_reader import read_monk, read_cup
from validation import kfold_cv
from sklearn.utils import shuffle
import numpy as np

In [ ]:
train_data, train_target = read_monk('datasets/monks/monks-1.train')
test_data, test_target = read_monk('datasets/monks/monks-1.test')
train_data, train_target = shuffle(train_data, train_target)

# split_index = len(train_data) * 2 // 3
# train_data, val_data = train_data[:split_index], train_data[split_index:]
# train_target, val_target = train_target[:split_index], train_target[split_index:]

In [ ]:
nn_monk1 = NeuralNetwork(
    n_hidden_layers=1, 
    hidden_layer_sizes=[3], 
    n_output_units=1, 
    training_loss_type_value=LossFunction.MSE.value, 
    validation_loss_type_value=LossFunction.MSE.value, 
    activation_hidden_type_value=ActivationFunction.SIGMOID.value,
    activation_output_type_value=ActivationFunction.SIGMOID.value, 
    learning_rate=0.2,
    reg_lambda=0,
    mom_alpha=0.9,
    epochs=300, 
    batch_size=1, 
    classification=True, 
    early_stopping=False, 
    patience=20,
    tollerance=0.0001, 
    verbose=False
    ).train_net(train_data, train_target)

In [ ]:
plot_over_epochs(y_values=nn_monk1.training_losses, title='training loss over epochs', y_label='loss', legend='MSE training loss')
plot_over_epochs(y_values=nn_monk1.training_accuracies, title='training accuracy over epochs', y_label='accuracy', legend='training accuracy')
# plot_over_epochs(y_values=nn_monk1.validation_losses, title='validation loss over epochs', y_label='loss', legend='MSE validation loss')
# plot_over_epochs(y_values=nn_monk1.validation_accuracies, title='validation accuracy over epochs', y_label='accuracy', legend='validation accuracy')

In [ ]:
test_accuracy = nn_monk1.predict_and_evaluate(test_data, test_target, Metrics.ACCURACY.value)
test_mse = nn_monk1.predict_and_evaluate(test_data, test_target, Metrics.MSE.value)
test_mee = nn_monk1.predict_and_evaluate(test_data, test_target, Metrics.MEE.value)
print("TEST RESULTS:")
print(f"Accuracy: {test_accuracy} \nMSE: {test_mse} \nMEE: {test_mee}")

In [ ]:
train_data, train_target = read_monk('datasets/monks/monks-2.train')
test_data, test_target = read_monk('datasets/monks/monks-2.test')
train_data, train_target = shuffle(train_data, train_target)

split_index = len(train_data) * 2 // 3
train_data, val_data = train_data[:split_index], train_data[split_index:]
train_target, val_target = train_target[:split_index], train_target[split_index:]

In [ ]:
nn_monk2 = NeuralNetwork(
    n_hidden_layers=1,
    hidden_layer_sizes=[3], 
    n_output_units=1, 
    training_loss_type_value=LossFunction.MSE.value, 
    validation_loss_type_value=LossFunction.MSE.value, 
    activation_hidden_type_value=ActivationFunction.SIGMOID.value, 
    activation_output_type_value=ActivationFunction.SIGMOID.value, 
    learning_rate=0.2,
    reg_lambda=0,
    mom_alpha=0.9,
    epochs=1000, 
    batch_size=1, 
    classification=True, 
    early_stopping=True, 
    patience=20, 
    tollerance=0.00001, 
    verbose=False
    ).train_net(train_data, train_target, test_data, test_target)

In [ ]:
plot_over_epochs(y_values=nn_monk2.training_losses, title='MSE loss over epochs', y_label='loss', legend='MSE training loss')
plot_over_epochs(y_values=nn_monk2.training_accuracies, title='training accuracy over epochs', y_label='accuracy', legend='training accuracy')
plot_over_epochs(y_values=nn_monk2.validation_losses, title='MSE validation loss over epochs', y_label='loss', legend='MSE validation loss')
plot_over_epochs(y_values=nn_monk2.validation_accuracies, title='validation accuracy over epochs', y_label='accuracy', legend='validation accuracy')

In [ ]:
test_accuracy = nn_monk2.predict_and_evaluate(test_data, test_target, Metrics.ACCURACY.value)
test_mse = nn_monk2.predict_and_evaluate(test_data, test_target, Metrics.MSE.value)
test_mee = nn_monk2.predict_and_evaluate(test_data, test_target, Metrics.MEE.value)
print("TEST RESULTS:")
print(f"Accuracy: {test_accuracy} \nMSE: {test_mse} \nMEE: {test_mee}")

In [ ]:
train_data, train_target = read_monk('datasets/monks/monks-3.train')
test_data, test_target = read_monk('datasets/monks/monks-3.test')
train_data, train_target = shuffle(train_data, train_target)

split_index = len(train_data) * 2 // 3
train_data, val_data = train_data[:split_index], train_data[split_index:]
train_target, val_target = train_target[:split_index], train_target[split_index:]

In [ ]:
nn_monk3 = NeuralNetwork(
    n_hidden_layers=1, 
    hidden_layer_sizes=[3], 
    n_output_units=1, 
    training_loss_type_value=LossFunction.MSE.value, 
    validation_loss_type_value=LossFunction.MSE.value, 
    activation_hidden_type_value=ActivationFunction.SIGMOID.value, 
    activation_output_type_value=ActivationFunction.SIGMOID.value, 
    learning_rate=0.02, 
    reg_lambda=0.0005,
    mom_alpha=0.9,
    epochs=1000, 
    batch_size=1, 
    classification=True, 
    early_stopping=True, 
    patience=60, 
    tollerance=0.001, 
    verbose=False
    ).train_net(train_data, train_target, test_data, test_target)

In [ ]:
plot_over_epochs(y_values=nn_monk3.training_losses, title='MSE loss over epochs', y_label='loss', legend='MSE training loss')
plot_over_epochs(y_values=nn_monk3.training_accuracies, title='training accuracy over epochs', y_label='accuracy', legend='training accuracy')
plot_over_epochs(y_values=nn_monk3.validation_losses, title='MSE validation loss over epochs', y_label='loss', legend='MSE validation loss')
plot_over_epochs(y_values=nn_monk3.validation_accuracies, title='validation accuracy over epochs', y_label='accuracy', legend='validation accuracy')

In [ ]:
test_accuracy = nn_monk3.predict_and_evaluate(test_data, test_target, Metrics.ACCURACY.value)
test_mse = nn_monk3.predict_and_evaluate(test_data, test_target, Metrics.MSE.value)
test_mee = nn_monk3.predict_and_evaluate(test_data, test_target, Metrics.MEE.value)
print("TEST RESULTS:")
print(f"Accuracy: {test_accuracy} \nMSE: {test_mse} \nMEE: {test_mee}")

In [ ]:
data, target = read_monk('datasets/monks/monks-3.train')

config = dict(
    n_hidden_layers=1, 
    hidden_layer_sizes=[3], 
    n_output_units=1, 
    training_loss_type_value=LossFunction.MSE.value, 
    validation_loss_type_value=LossFunction.MSE.value, 
    activation_hidden_type_value=ActivationFunction.SIGMOID.value, 
    activation_output_type_value=ActivationFunction.SIGMOID.value, 
    learning_rate=0.02, 
    reg_lambda=0.0005,
    mom_alpha=0.9,
    epochs=1000, 
    batch_size=1, 
    classification=True, 
    early_stopping=True, 
    patience=60, 
    tollerance=0.001, 
    verbose=False
)

metrics = [Metrics.MSE.value, Metrics.MEE.value, Metrics.ACCURACY.value]
kfold_cv(10, data, target, metrics, cv_verbose=True, **config)

In [ ]:
from validation import splitter
data, targets = read_cup('datasets/cup/CUP_TR.csv')
train_data, test_data, train_target, test_target = splitter(data, targets, 0.8)

config = dict(
    n_hidden_layers=3, 
    hidden_layer_sizes=[10, 10, 10], 
    n_output_units=3, 
    training_loss_type_value=LossFunction.MSE.value, 
    validation_loss_type_value=LossFunction.MSE.value, 
    activation_hidden_type_value=ActivationFunction.SIGMOID.value, 
    activation_output_type_value=ActivationFunction.SIGMOID.value, 
    learning_rate=0.02, 
    reg_lambda=0.0005,
    mom_alpha=0.9,
    epochs=1000, 
    batch_size=1, 
    classification=False, 
    early_stopping=True, 
    patience=20, 
    tollerance=0.001, 
    verbose=False
)

metrics = [Metrics.MSE.value, Metrics.MEE.value]
nn, results = kfold_cv(10, train_data, train_target, metrics, cv_verbose=True, **config)

In [ ]:
nn.predict_and_evaluate(test_data, test_target, Metrics.MSE.value)

In [ ]:
results